<a href="https://colab.research.google.com/github/Sangramjit786/Sangramjit786.github.io/blob/master/Question%26answer_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers==3.0.0

     |████████████████████████████████| 757kB 6.6MB/s 
     |████████████████████████████████| 1.2MB 11.7MB/s 
     |████████████████████████████████| 3.0MB 38.7MB/s 
     |████████████████████████████████| 901kB 35.8MB/s 


In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-04-24 04:49:31--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M  73.7MB/s    in 0.5s    

2021-04-24 04:49:32 (73.7 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-04-24 04:49:32--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

sq

In [4]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
        print(squad_dict['data'][0])
    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                  access= 'plausible_answer'
                else:
                  access= 'answers'
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    return contexts, questions, answers

In [5]:
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

{'title': 'Normans', 'paragraphs': [{'qas': [{'question': 'In what country is Normandy located?', 'id': '56ddde6b9a695914005b9628', 'answers': [{'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}], 'is_impossible': False}, {'question': 'When were the Normans in Normandy?', 'id': '56ddde6b9a695914005b9629', 'answers': [{'text': '10th and 11th centuries', 'answer_start': 94}, {'text': 'in the 10th and 11th centuries', 'answer_start': 87}, {'text': '10th and 11th centuries', 'answer_start': 94}, {'text': '10th and 11th centuries', 'answer_start': 94}], 'is_impossible': False}, {'question': 'From which countries did the Norse originate?', 'id': '56ddde6b9a695914005b962a', 'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256}, {'text': 'Denmark, Iceland and Norway', 'answer_start': 256}, {'text': 'Denmark, Iceland and Norway', 'answer_start': 256}, {'text': 'Denm

In [6]:
'''def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
 
        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        else:
          for n in [1, 2]:
            if context[start_idx-n:end_idx-n] == gold_text:
              answer['answer_start'] = start_idx - n
              answer['answer_end'] = end_idx - n
 
#add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)'''

"def add_end_idx(answers, contexts):\n    for answer, context in zip(answers, contexts):\n        gold_text = answer['text']\n        start_idx = answer['answer_start']\n        end_idx = start_idx + len(gold_text)\n \n        # sometimes squad answers are off by a character or two – fix this\n        if context[start_idx:end_idx] == gold_text:\n            answer['answer_end'] = end_idx\n        else:\n          for n in [1, 2]:\n            if context[start_idx-n:end_idx-n] == gold_text:\n              answer['answer_start'] = start_idx - n\n              answer['answer_end'] = end_idx - n\n \n#add_end_idx(train_answers, train_contexts)\nadd_end_idx(val_answers, val_contexts)"

In [7]:
length=len(val_answers)
print(length)

20302


In [8]:
'''encoding = tokenizer.encode_plus(text=val_questions[5544],text_pair=val_contexts[5544])
inputs = encoding['input_ids'] 
sentence_embedding = encoding['token_type_ids']  
start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
tokens = tokenizer.convert_ids_to_tokens(inputs) 
answer = ' '.join(tokens[start_index:end_index+1])
corrected_answer = ''
 
for word in answer.split():
    
    
  if word[0:2] == '##':
      corrected_answer += word[2:]
  else:
      corrected_answer += ' ' + word
corrected_answer=corrected_answer.lstrip()
corrected_answer=corrected_answer.rstrip()
print(val_questions[5544])
print("correct answer: "+val_answers[5544]['text'].lower())
print("predicted: "+corrected_answer)'''

'encoding = tokenizer.encode_plus(text=val_questions[5544],text_pair=val_contexts[5544])\ninputs = encoding[\'input_ids\'] \nsentence_embedding = encoding[\'token_type_ids\']  \nstart_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))\nstart_index = torch.argmax(start_scores)\nend_index = torch.argmax(end_scores)\ntokens = tokenizer.convert_ids_to_tokens(inputs) \nanswer = \' \'.join(tokens[start_index:end_index+1])\ncorrected_answer = \'\'\n \nfor word in answer.split():\n    \n    \n  if word[0:2] == \'##\':\n      corrected_answer += word[2:]\n  else:\n      corrected_answer += \' \' + word\ncorrected_answer=corrected_answer.lstrip()\ncorrected_answer=corrected_answer.rstrip()\nprint(val_questions[5544])\nprint("correct answer: "+val_answers[5544][\'text\'].lower())\nprint("predicted: "+corrected_answer)'

In [9]:
acc=[]
question=val_questions[0:2304]
context=val_contexts[0:2304]
answers=val_answers[0:2304]
length=len(context)
print(length)

2304


In [10]:
for i in range(length):
  print(i)
  encoding = tokenizer.encode_plus(text=val_questions[i],text_pair=val_contexts[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
  #print(val_questions[i])
  #print("correct answer: "+val_answers[i]['text'].lower())
  #print("predicted: "+corrected_answer)
  acc.append(corrected_answer==val_answers[i]['text'].lower())

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152


In [11]:
print(sum(acc))
print(len(acc))
sum(acc)/len(acc)


1259
2304


0.5464409722222222

In [12]:
question=val_questions[2304:4001]
context=val_contexts[2304:4001]
answers=val_answers[2304:4001]
length1=len(context)
print(length1)

1697


In [13]:
for i in range(length1):
  encoding = tokenizer.encode_plus(text=val_questions[i],text_pair=val_contexts[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
  #print(val_questions[i])
  #print("correct answer: "+val_answers[i]['text'].lower())
  #print("predicted: "+corrected_answer)
  acc.append(corrected_answer==val_answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
print(sum(acc))
print(len(acc))
sum(acc)/len(acc)

2216
4001


0.5538615346163459

In [15]:
#acc1 = []
question=val_questions[4001:6000]
context=val_contexts[4001:6000]
answers=val_answers[4001:6000]
length2=len(context)
print(length2)


1999


In [16]:
for i in range(length2):
  encoding = tokenizer.encode_plus(text=val_questions[i],text_pair=val_contexts[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
  #print(val_questions[i])
  #print("correct answer: "+val_answers[i]['text'].lower())
  #print("predicted: "+corrected_answer)
  acc.append(corrected_answer==val_answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
print(sum(acc))
print(len(acc))
sum(acc)/len(acc)

3319
6000


0.5531666666666667

In [18]:
question=val_questions[6000:8000]
context=val_contexts[6000:8000]
answers=val_answers[6000:8000]
length3=len(context)
print(length3)

2000


In [19]:
for i in range(length3):
  encoding = tokenizer.encode_plus(text=val_questions[i],text_pair=val_contexts[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
  #print(val_questions[i])
  #print("correct answer: "+val_answers[i]['text'].lower())
  #print("predicted: "+corrected_answer)
  acc.append(corrected_answer==val_answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
print(sum(acc))
print(len(acc))
sum(acc)/len(acc)

4422
8000


0.55275

In [21]:
question=val_questions[8000:10001]
context=val_contexts[8000:10001]
answers=val_answers[8000:10001]
length4=len(context)
print(length4)

2001


In [22]:
for i in range(length4):
  encoding = tokenizer.encode_plus(text=val_questions[i],text_pair=val_contexts[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
  #print(val_questions[i])
  #print("correct answer: "+val_answers[i]['text'].lower())
  #print("predicted: "+corrected_answer)
  acc.append(corrected_answer==val_answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
print(sum(acc))
print(len(acc))
sum(acc)/len(acc)

5525
10001


0.5524447555244476

In [24]:
acc1 = []
question=val_questions[10001:12001]
context=val_contexts[10001:12001]
answers=val_answers[10001:12001]
length5=len(context)
print(length5)

2000


In [25]:

for i in range(length5):
  encoding = tokenizer.encode_plus(text=question[i],text_pair=context[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
 # print(str(i)+")"+question[i])
  #print("correct answer: "+answers[i]['text'].lower())
 # print("predicted: "+corrected_answer)
  acc1.append(corrected_answer==answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
print(sum(acc1))
print(len(acc1))
sum(acc1)/len(acc1)

1068
2000


0.534

In [27]:

question=val_questions[12001:14001]
context=val_contexts[12001:14001]
answers=val_answers[12001:14001]
length6=len(context)
print(length6)

2000


In [28]:
for i in range(length6):
  encoding = tokenizer.encode_plus(text=question[i],text_pair=context[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
 # print(str(i)+")"+question[i])
  #print("correct answer: "+answers[i]['text'].lower())
 # print("predicted: "+corrected_answer)
  acc1.append(corrected_answer==answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [29]:
print(sum(acc1))
print(len(acc1))
sum(acc1)/len(acc1)

2275
4000


0.56875

In [30]:
question=val_questions[14001:16001]
context=val_contexts[14001:16001]
answers=val_answers[14001:16001]
length7=len(context)
print(length7)

2000


In [31]:
for i in range(length7):
  encoding = tokenizer.encode_plus(text=question[i],text_pair=context[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
 # print(str(i)+")"+question[i])
  #print("correct answer: "+answers[i]['text'].lower())
 # print("predicted: "+corrected_answer)
  acc1.append(corrected_answer==answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
print(sum(acc1))
print(len(acc1))
sum(acc1)/len(acc1)

3231
6000


0.5385

In [33]:
question=val_questions[16001:18001]
context=val_contexts[16001:18001]
answers=val_answers[16001:18001]
length8=len(context)
print(length8)

2000


In [34]:
for i in range(length8):
  encoding = tokenizer.encode_plus(text=question[i],text_pair=context[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
 # print(str(i)+")"+question[i])
  #print("correct answer: "+answers[i]['text'].lower())
 # print("predicted: "+corrected_answer)
  acc1.append(corrected_answer==answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
print(sum(acc1))
print(len(acc1))
sum(acc1)/len(acc1)

4403
8000


0.550375

In [36]:
question=val_questions[18001:]
context=val_contexts[18001:]
answers=val_answers[18001:]
length9=len(answers)
print(length9)

2301


In [37]:
for i in range(length9):
  encoding = tokenizer.encode_plus(text=question[i],text_pair=context[i])
  inputs = encoding['input_ids'] 
  sentence_embedding = encoding['token_type_ids']  
  start_scores, end_scores = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  tokens = tokenizer.convert_ids_to_tokens(inputs) 
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_answer = ''
 
  for word in answer.split():
    
    
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
  corrected_answer=corrected_answer.lstrip()
  corrected_answer=corrected_answer.rstrip()
  print(str(i))
 # print(str(i)+")"+question[i])
  #print("correct answer: "+answers[i]['text'].lower())
 # print("predicted: "+corrected_answer)
  acc1.append(corrected_answer==answers[i]['text'].lower())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
print(sum(acc1))
print(len(acc1))
sum(acc1)/len(acc1)

5574
10301


0.5411125133482186

In [40]:
accuracy = acc + acc1
print(sum(accuracy))
print(len(accuracy))
print(round((sum(accuracy)/len(accuracy))*100  ))

11099
20302
55
